## 


In [100]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import random

In [106]:
def get_info(title, author):
    '''Given a title and author of a book in the list,
    returns a dictionary of prosecraft's analysis about the book.'''
    info = {'title': title, 'author': author}
    title = title.replace(' ', '-').replace(':','').lower()
    author = author.replace(' ', '-').lower()
    page = requests.get(f"http://prosecraft.io/library/{author}/{title}/")
    soup = BeautifulSoup(page.content, "html.parser")
    headings = soup.find_all("div", {"class": "book-info-metric-heading"})
    values = soup.find_all("div", {"class": "book-info-metric-value"})
    for heading, value in zip(headings, values):
        info[heading.text] = float(value.text.strip('%').replace(',',''))
    return info

In [64]:
get_info("Thick as Thieves", "Megan Whalen Turner")

{'title': 'Thick as Thieves',
 'author': 'Megan Whalen Turner',
 'total words': 81228.0,
 'vividness': 46.07,
 'passive voice': 8.24,
 'all adverbs': 2.9,
 'ly-adverbs': 1.0,
 'non-ly-adverbs': 1.91}

In [60]:
sample_book = Book("Thick as Thieves", "Megan Whalen Turner")

In [61]:
sample_book.non_ly_adverbs

1.91

In [72]:
books_table = pd.DataFrame([get_info("Thick as Thieves", "Megan Whalen Turner"),get_info("Return of the Thief", "Megan Whalen Turner")])

In [94]:
with open('book_list.json', 'r') as lst:
    book_list = json.load(lst)


24997


In [95]:
indices = random.sample(range(0, 24997), 10)
books = []
for index in indices:
    books.append(book_list[index])

In [96]:
books

[{'a': 'Adam Mitzner',
  't': 'The Perfect Marriage',
  'w': 200,
  'e': 'jpg',
  'v': 80620.0},
 {'a': 'Menna van Praag',
  't': 'The Sisters Grimm',
  'w': 198,
  'e': 'jpg',
  'v': 127292.0},
 {'a': 'Dylan Rodriguez',
  't': 'White Reconstruction',
  'w': 200,
  'e': 'jpg',
  'v': 69519.0},
 {'a': 'Lauren Blakely',
  't': 'Scoring with Him',
  'w': 189,
  'e': 'jpg',
  'v': 79455.0},
 {'a': 'Dugald Bruce-Lockhart',
  't': 'The Lizard',
  'w': 197,
  'e': 'jpg',
  'v': 78250.0},
 {'a': 'Jennifer Bacia',
  't': 'Never Forget Me',
  'w': 196,
  'e': 'jpg',
  'v': 83058.0},
 {'a': 'Anthony Riches',
  't': 'Empire: Fortress of Spears',
  'w': 196,
  'e': 'jpg',
  'v': 123371.0},
 {'a': 'Liliiana Saint Clair',
  't': 'Trap for a Goose',
  'w': 189,
  'e': 'jpg',
  'v': 45932.0},
 {'a': 'Metis Beach',
  't': 'Claudine Bourbonnais',
  'w': 195,
  'e': 'jpg',
  'v': 130572.0},
 {'a': 'Roy Peter Clark',
  't': 'The Art of X-Ray Reading',
  'w': 198,
  'e': 'jpg',
  'v': 68348.0}]

In [108]:
sample = []

for book in books:
    sample.append(get_info(book['t'], book['a']))
    

In [109]:
df = pd.DataFrame(sample)

In [110]:
df

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs
0,The Perfect Marriage,Adam Mitzner,80620.0,24.66,9.95,3.23,1.03,2.20
1,The Sisters Grimm,Menna van Praag,127292.0,62.51,8.52,3.40,0.91,2.48
2,White Reconstruction,Dylan Rodriguez,69519.0,29.69,3.57,2.79,1.68,1.11
3,Scoring with Him,Lauren Blakely,79455.0,49.74,7.69,2.99,0.77,2.22
4,The Lizard,Dugald Bruce-Lockhart,78250.0,60.19,6.17,2.54,0.76,1.79
5,Never Forget Me,Jennifer Bacia,83058.0,37.02,8.66,3.29,1.17,2.12
6,Empire: Fortress of Spears,Anthony Riches,123371.0,47.48,5.90,2.61,1.09,1.52
7,Trap for a Goose,Liliiana Saint Clair,45932.0,32.85,7.30,4.11,1.80,2.31
8,Claudine Bourbonnais,Metis Beach,130572.0,44.49,7.52,2.70,0.91,1.79
9,The Art of X-Ray Reading,Roy Peter Clark,68348.0,43.19,5.72,2.01,0.60,1.41
